hotel: The type of hotel, either "City Hotel" or "Resort Hotel.", resort = vacances

is_canceled: Binary value indicating whether the booking was cancelled (1) or not (0).

lead_time: Number of days between booking and arrival.

arrival_date_year: Year of arrival date.

arrival_date_month: Month of arrival date.

arrival_date_week_number: Week number of arrival date.

arrival_date_day_of_month: Day of the month of arrival date.

stays_in_weekend_nights: Number of weekend nights (Saturday or Sunday) the guest stays.

stays_in_week_nights: Number of weekday nights (Monday to Friday) the guest stays.

adults: Number of adults.

children: Number of children.

babies: Number of babies.

meal: Type of meal booked.

country: Country of origin.

market_segment: Market segment designation.

distribution_channel: Booking distribution channel.

is_repeated_guest: Binary value indicating whether the guest is a repeated guest (1) or not (0).

previous_cancellations: Number of previous booking cancellations.

previous_bookings_not_canceled: Number of previous bookings not cancelled.

reserved_room_type: Code of room type reserved.

assigned_room_type: Code of room type assigned at check-in.

booking_changes: Number of changes/amendments made to the booking.

deposit_type: Type of deposit made.

agent: ID of the travel agency.

company: ID of the company.

days_in_waiting_list: Number of days in the waiting list before booking.

customer_type: Type of booking.

adr: Average daily rate. combien gagne l'hotel, pour la journée donnée, par chambre

required_car_parking_spaces: Number of car parking spaces required.

total_of_special_requests: Number of special requests made.

reservation_status: Reservation last status.

reservation_status_date: Date of the last status.

name: Guest's name. (Not Real)

email: Guest's email address.(Not Real)

phone-number: Guest's phone number. (Not Real)

credit_card: Guest's credit card details. (Not Real)

In [108]:
import pandas as pd
import numpy as np 
import matplotlib as plt

In [109]:
df = pd.read_csv("../hotel_bookings.csv", sep=",")
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,...,No Deposit,394.0,NaN,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,...,No Deposit,9.0,NaN,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,...,No Deposit,9.0,NaN,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,...,No Deposit,89.0,NaN,0,Transient,104.40,0,0,Check-Out,2017-09-07


# Cleaning

### NaN and select columns

In [110]:
colonne_a_garder = ["hotel", "is_canceled", "lead_time", "arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "stays_in_weekend_nights", "stays_in_week_nights", "adults", "children", "babies", "meal", "country", "market_segment", "is_repeated_guest", "previous_cancellations", "previous_bookings_not_canceled", "reserved_room_type", "assigned_room_type", "booking_changes", "deposit_type", "days_in_waiting_list", "customer_type", "adr", "required_car_parking_spaces", "total_of_special_requests"]
df = df[colonne_a_garder]
df.dropna(inplace=True)
df.drop_duplicates(subset=colonne_a_garder, inplace=True)
print(df.isnull().sum())




C:\Users\paulp\AppData\Local\Temp\ipykernel_4160\4157271603.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\paulp\AppData\Local\Temp\ipykernel_4160\4157271603.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=colonne_a_garder, inplace=True)


hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
days_in_waiting_list              0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
dtype: int64


## type of columns (date, ...)

In [111]:
df['arrival_date_month'] = pd.to_datetime(df.arrival_date_month, format='%B', errors='coerce').dt.month
df['arrival_date'] = pd.to_datetime(df[['arrival_date_year','arrival_date_month','arrival_date_day_of_month']].astype(str).agg('-'.join, axis=1))
df.drop(['arrival_date_year','arrival_date_month','arrival_date_day_of_month'], axis=1, inplace=True)
print(df)

C:\Users\paulp\AppData\Local\Temp\ipykernel_4160\2894240059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['arrival_date_month'] = pd.to_datetime(df.arrival_date_month, format='%B', errors='coerce').dt.month


               hotel  is_canceled  lead_time  stays_in_weekend_nights  \
0       Resort Hotel            0        342                        0   
1       Resort Hotel            0        737                        0   
2       Resort Hotel            0          7                        0   
3       Resort Hotel            0         13                        0   
4       Resort Hotel            0         14                        0   
...              ...          ...        ...                      ...   
119385    City Hotel            0         23                        2   
119386    City Hotel            0        102                        2   
119387    City Hotel            0         34                        2   
119388    City Hotel            0        109                        2   
119389    City Hotel            0        205                        2   

        stays_in_week_nights  adults  children  babies meal country  ...  \
0                          0       2       0.0 

C:\Users\paulp\AppData\Local\Temp\ipykernel_4160\2894240059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['arrival_date'] = pd.to_datetime(df[['arrival_date_year','arrival_date_month','arrival_date_day_of_month']].astype(str).agg('-'.join, axis=1))
C:\Users\paulp\AppData\Local\Temp\ipykernel_4160\2894240059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['arrival_date_year','arrival_date_month','arrival_date_day_of_month'], axis=1, inplace=True)


## nettoyage données valeurs aberrantes (var quanti), modalité seule (var quali)

In [112]:

# Quantitative variables
occurrences = df['lead_time'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['lead_time'].isin(values_to_keep)]

occurrences = df['stays_in_weekend_nights'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['stays_in_weekend_nights'].isin(values_to_keep)]

occurrences = df['stays_in_week_nights'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['stays_in_week_nights'].isin(values_to_keep)]

occurrences = df['adults'].value_counts()
values_to_keep = occurrences[occurrences >= 5].index
df = df[df['adults'].isin(values_to_keep)]

occurrences = df['children'].value_counts()
values_to_keep = occurrences[occurrences >= 2].index
df = df[df['children'].isin(values_to_keep)]

occurrences = df['babies'].value_counts()
values_to_keep = occurrences[occurrences >= 2].index
df = df[df['babies'].isin(values_to_keep)]

occurrences = df['previous_cancellations'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['previous_cancellations'].isin(values_to_keep)]

occurrences = df['previous_bookings_not_canceled'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['previous_bookings_not_canceled'].isin(values_to_keep)]

occurrences = df['booking_changes'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['booking_changes'].isin(values_to_keep)]

occurrences = df['days_in_waiting_list'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['days_in_waiting_list'].isin(values_to_keep)]

occurrences = df['adr'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['adr'].isin(values_to_keep)]

occurrences = df['required_car_parking_spaces'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['required_car_parking_spaces'].isin(values_to_keep)]

# Qualitative variables
occurrences = df['reserved_room_type'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['reserved_room_type'].isin(values_to_keep)]

occurrences = df['assigned_room_type'].value_counts()
values_to_keep = occurrences[occurrences >= 10].index
df = df[df['assigned_room_type'].isin(values_to_keep)]

df = df[df['meal'] != "Undefined"]

## binary to categorielle

In [114]:
columns_binary = ["hotel", "is_canceled", "is_repeated_guest"]

for col in df.columns:
    print(col, df[col].unique())


hotel ['Resort Hotel' 'City Hotel']
is_canceled [0 1]
lead_time [342   7  13  14   0   9  85  75  23  35  68  37  12 127  48  60  77  99
  96  69  45  40  15  36  43  70  16 107  47 113  90  76   3   1  10   5
  17  93  51  63  62 101   2  72 364  81  21 109 102  98  95  26  73 115
  86  78  29  92  79  30  33  32   8 100  44  80  97  64  39  34  27 110
 111  84  66 104  28  94 112  65  67  55  88  54  83 105 280  82  24 249
  22  91  11 108 106  31 304 117  71  53  59 116  50  42  58  38   6  57
  19 315  25 123  87   4 118  41  52  49  61  89  18 299  74  20  46  56
 286 136 129 124 130 327 131 140 114 139 122 126 120 128 135 150 151 132
 157 147 138 156 103 164 143 346 160 333 149 154 297 155 323 340 248 175
 344 146 159 166 310 172 145 121 170 178 305 152 167 119 158 125 185 183
 165 200 207 174 330 134 350 283 153 197 133 163 148 171 241 193 192 209
 187 223 169 284 173 142 312 230 161 188 144 219 162 196 190 259 238 141
 168 182 186 210 275 214 232 218 206 208 191 181 229 189 179